In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
print(OpenAI.api_key)

In [ ]:
instructions = [
    "# Objective: ",
    "As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively." ,
    "# Main features of the Flora app: ",
    "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.",
    "# Support Process: ",
    "1. Issue Identification: To ensure that I understand your problems, I will summarize the constructive, non-emotional parts of your input text in your tone into one or more key points. Then, I will and must list these key points in bullet form starting with '-' and confirm with you if they are correct.",
    "2. Verification & Refinement: If you have following-up clarification for any key point, I'll refine the key points by considering your clarification. I will also identify new key points and add them into the refined list.",
    "3. Repeat: I'll repeat the previous two steps until you're satisfied. "
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

In [ ]:
import csv

with open('./data/manual.csv') as file:
    reader = csv.reader(file)
    data = list(reader)

data[0].append('Key_Point')

In [ ]:
for i in range(1,670):

    question = data[i][0]

    completion1 = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": question}
        ]
    )
    time.sleep(3)

    completion2 = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "Retain only the key points which is a sentence starting with '-'. Ensure that the retained key points remain unchanged and intact. Delete any content that is not included in the key points."},
            {"role": "user", "content": "Retain only the key points which is a sentence starting with '-'. Ensure that the retained key points remain unchanged and intact. Delete any content that is not included in the key points: \n" + completion1.choices[0].message.content}
        ]
    )
    time.sleep(3)

    data[i].append(completion2.choices[0].message.content.replace("\n", " "))

with open('./QAK.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [ ]:
import pandas as pd

def get_text_embeddings(csv_file, max_length=8191):
    df = pd.read_csv(csv_file)
    embeddings = []

    for index, row in df.iterrows():
        
        question = row['Question']
        answer = row['Answer']
        key_point = row['Key_Point']

        # Split the text into tokens
        tokens = key_point.split()

        # Truncate or pad the tokens to the maximum length
        tokens = tokens[:max_length]

        # Join the tokens back into a single string
        truncated_text = ' '.join(tokens)

        # Get the text embeddings using OpenAI
        embedding = client.embeddings.create(input=[truncated_text], model="text-embedding-3-large", dimensions=3072).data[0].embedding
        time.sleep(3)
        
        # Append the embedding to the list
        embeddings.append(embedding)

    # Create a new DataFrame with the embeddings
    embeddings_df = pd.DataFrame(embeddings)

    return embeddings_df

In [ ]:
embeddings_df = get_text_embeddings('QAK.csv')
embeddings_df.to_csv('./data/key_point_embeddings.csv', index=False)